# Laboratorio 4 - Statistical Learning I - Universidad Galileo

#### Importación de librerías

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler

#### Importación de dataset

In [80]:
dataset = pd.read_csv('train_HP.csv')

#### Funciones necesarias

In [6]:
def GetNanColumnsNames(df):
    colnames = df.columns
    cols_con_na = []
    for col in colnames:
        if(df[col].isnull().sum() > 0):
            cols_con_na.append(col)
    return cols_con_na

In [7]:
def GetNanGoodColumnsNames(df, rate = 0.2):
    cols_procesables = []
    for col in df.columns:
        if((df[col].isnull().mean() < rate)):
            cols_procesables.append(col)
    return cols_procesables

In [24]:
def GetNumericColumnsNames(df):
    colnames = df.columns
    nums_cols_na = []
    for col in colnames:
        if(((df[col].dtypes == 'int64') or (df[col].dtypes == 'float64'))):
            nums_cols_na.append(col)
    return nums_cols_na

In [5]:
def ImputacionPorMediaOMediana(dataset, columna, target):
    imputacionMedia = np.round(dataset[columna].mean(), 0)
    imputacionMediana = np.round(dataset[columna].median(), 0)
    datasetColumnaImputacionMedia = dataset[columna].fillna(imputacionMedia)
    datasetColumnaImputacionMediana = dataset[columna].fillna(imputacionMediana)
    
    proporcion = dataset[columna].isnull().mean()
    
    if(proporcion <= .2):
        correlacionMedia = np.corrcoef(datasetColumnaImputacionMedia, dataset[target])[0,1]
        correlacionMediana = np.corrcoef(datasetColumnaImputacionMediana, dataset[target])[0,1]

        if(correlacionMedia >= correlacionMediana):
            dataset[columna] = dataset[columna].fillna(imputacionMedia)
        else:
            dataset[columna] = dataset[columna].fillna(imputacionMediana) 

In [87]:
def GetCategoricalColumnsNames(df):
    colnames = df.columns
    cat_cols = []
    for col in colnames:
        if(df[col].dtype == 'object'):
            cat_cols.append(col)
    return cat_cols

In [94]:
def GetContinuesColumnsNames(df):
    colnames = df.columns
    numeric_continues_vars = []
    for col in colnames:
        unique_values =len (df[col].unique())
        if((df[col].dtype != 'object') and (unique_values > 30)):
            numeric_continues_vars.append(col)
    return numeric_continues_vars

In [10]:
def ExecuteFrequencyEncoding(df, map_type='freq'):
    colnames = df.columns
    for col in colnames:
        if(df[col].dtype == 'object'):
            factor_div = 1 if (map_type == 'freq') else len(df[col])
            mapper = (df[col].value_counts().sort_values(ascending=False)/factor_div).to_dict()
            df[col] = df[col].map(mapper) 

In [12]:
def DetectOutliers(df, variable, factor):
    IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25)
    LI = df[variable].quantile(0.25) - (IQR*factor)
    LS = df[variable].quantile(0.75) + (IQR*factor)
    
    return LI, LS

In [13]:
def OutlierTreatment(df, variable, factor):
    IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25)
    LI = df[variable].quantile(0.25) - (IQR*factor)
    LS = df[variable].quantile(0.75) + (IQR*factor)
    
    df[variable] = np.where(df[variable] > LS, LS,
                                          np.where(df[variable] < LI, LI, df[variable])) 

## Tipos de variables

In [14]:
dataset.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object

In [15]:
columnasConNaN = GetNanColumnsNames(dataset)
dataset[columnasConNaN].dtypes

LotFrontage     float64
Alley            object
MasVnrType       object
MasVnrArea      float64
BsmtQual         object
BsmtCond         object
BsmtExposure     object
BsmtFinType1     object
BsmtFinType2     object
Electrical       object
FireplaceQu      object
GarageType       object
GarageYrBlt     float64
GarageFinish     object
GarageQual       object
GarageCond       object
PoolQC           object
Fence            object
MiscFeature      object
dtype: object

In [23]:
columnasContinuas = GetContinuesColumns(dataset)
columnasContinuas

['Id',
 'LotFrontage',
 'LotArea',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'GrLivArea',
 'GarageYrBlt',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 'ScreenPorch',
 'SalePrice']

In [20]:
columnasCategorias = GetCategoricalColumns(dataset)
columnasCategorias

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [26]:
columnasNumericas = GetNumericColumnsNames(dataset)
columnasNumericas

['Id',
 'MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'SalePrice']

## Imputación De Variables

In [81]:
columnasCCA = GetNanGoodColumnsNames(dataset)
columnasCCA 

['Id',
 'MSSubClass',
 'MSZoning',
 'LotFrontage',
 'LotArea',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3Ssn

In [82]:
dataset = dataset[columnasCCA].dropna()
GetNanGoodColumnsNames(dataset)

['Id',
 'MSSubClass',
 'MSZoning',
 'LotFrontage',
 'LotArea',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3Ssn

In [83]:
columnasNumericasConNaN = GetNanColumnsNames(dataset[columnasNumericas])

for columna in columnasNumericasConNaN:
    ImputacionPorMediaOMediana(dataset, columna, "SalePrice")

In [72]:
GetNanColumnsNames(dataset[columnasNumericas])

[]

In [85]:
dataset["BsmtQual"].fillna("Missing", inplace = True)


In [90]:
columnasCategoricas = GetCategoricalColumnsNames(dataset)
columnasCategoricas

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [91]:
pd.get_dummies(dataset['MSZoning'])

,C (all),FV,RH,RL,RM
0,0,0,0,1,0
1,0,0,0,1,0
2,0,0,0,1,0
3,0,0,0,1,0
4,0,0,0,1,0
...,...,...,...,...,...
1455,0,0,0,1,0
1456,0,0,0,1,0
1457,0,0,0,1,0
1458,0,0,0,1,0


In [92]:
ExecuteFrequencyEncoding(dataset)

## Transformación Variables Númericas

In [95]:
columnasContinuas = GetContinuesColumnsNames(dataset)
columnasContinuas

['Id',
 'LotFrontage',
 'LotArea',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'GrLivArea',
 'GarageYrBlt',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 'ScreenPorch',
 'SalePrice']

In [99]:
dataset["LotArea"] = np.log(dataset["LotArea"])

In [100]:
dataset["YearBuilt"] = (1 / dataset["YearBuilt"])

In [101]:
dataset['YearRemodAdd'] = (dataset['YearRemodAdd']**2)

In [105]:
dataset["TotalBsmtSF"], lambdaX = stats.boxcox(dataset["TotalBsmtSF"])

In [106]:
dataset["MasVnrArea"], lambdaX = stats.yeojohnson(dataset["MasVnrArea"])

## Outliers

In [110]:
for columna in columnasContinuas:
    OutlierTreatment(dataset, columna, 1.75)

## Feature Scaling

In [111]:
scaler = StandardScaler()

In [112]:
scaler.fit(dataset.loc[:, columnasContinuas]) 

StandardScaler()

In [117]:
dataset.loc[:, columnasContinuas] = pd.DataFrame(scaler.transform(dataset.loc[:, columnasContinuas]), columns=dataset.loc[:, columnasContinuas].columns)

In [118]:
np.round(dataset.loc[:, columnasContinuas].describe(), 2)

,Id,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageYrBlt,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch,SalePrice
count,821.00,821.00,821.00,821.00,821.00,821.00,821.00,821.0,821.00,821.00,821.00,821.00,821.00,821.00,821.00,821.00,821.00,821.0,821.0,821.00
mean,-0.01,0.00,-0.01,-0.01,0.01,0.01,0.03,0.0,-0.02,0.02,0.00,0.02,0.02,0.01,0.03,0.01,0.01,0.0,0.0,0.01
std,0.99,1.02,1.01,1.01,1.00,1.00,1.01,0.0,1.00,1.00,1.00,1.01,1.01,1.00,1.02,0.99,1.02,0.0,0.0,1.00
min,-1.73,-2.22,-2.39,-1.19,-1.71,-0.84,-1.01,0.0,-1.37,-3.20,-1.98,-0.81,-2.26,-3.03,-1.83,-0.80,-0.82,0.0,0.0,-2.05
25%,-0.84,-0.49,-0.59,-0.97,-0.86,-0.84,-1.01,0.0,-0.79,-0.69,-0.75,-0.81,-0.71,-0.72,-0.72,-0.80,-0.82,0.0,0.0,-0.71
50%,-0.02,0.01,-0.14,-0.13,0.48,-0.84,-0.10,0.0,-0.22,-0.02,-0.18,-0.81,-0.08,0.21,-0.08,-0.69,-0.30,0.0,0.0,-0.24
75%,0.84,0.60,0.42,0.61,0.91,1.17,0.65,0.0,0.54,0.78,0.65,0.87,0.56,0.94,0.60,0.67,0.47,0.0,0.0,0.55
max,1.74,2.24,2.25,3.06,1.16,1.54,3.43,0.0,2.84,3.26,3.12,3.76,2.78,1.21,2.80,3.21,2.67,0.0,0.0,2.64
